In [1]:
# What affects backflow the most: temperature or pressure
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random

In [2]:
# get all csv and store as pandas dataframe
chunk = pd.read_csv('../data/backflow_data.csv',chunksize=1001, sep=",")
pd_df = pd.concat(chunk)
df = pd.DataFrame(pd_df)

In [3]:
# quick sort implementation
def quickSort(arr, ascending=True):
    if arr == []:
        return arr
    smaller, equal, larger = [], [], []
    pivot = arr[random.randint(0, len(arr) - 1)]
    for x in arr:
        if x < pivot: smaller.append(x)
        elif x == pivot: equal.append(x)
        else: larger.append(x)
    larger = quickSort(larger, ascending=ascending)
    smaller = quickSort(smaller, ascending=ascending)
    if ascending:
        final = smaller + equal + larger
    else:
        final = larger + equal + smaller
    return final

In [4]:
# return avgs of data for each interval
# adjusted to replicate data from spreadsheet
def getAvgData(data, interval, isAscending=True):
    avgs = []
    desiredInterval = interval
    counter=0
    intervalCounter = 0
    total = 0
    for temp in data:
        total = total + float(temp.split("%")[0])
        counter = counter +1
        if intervalCounter ==0: interval = 192
        else: interval = desiredInterval
        # avg temperatures for each n rows
        if counter % interval == 0:
            avg = total / counter 
            avgs.append(avg)
            avg = 0
            counter = 0
            total = 0 
        intervalCounter = intervalCounter  +1
    if isAscending:
        return pd.DataFrame(quickSort(avgs, ascending=True))
    return pd.DataFrame(quickSort(avgs, ascending=False))

In [5]:
# avg of avg backflows (last column in provided excel sheet)
def getAvgAvgBackflows(avgBackflows, interval, isAscending=True):
    avgs = []
    counter = 0
    total = 0
    for avgbackflow in avgBackflows:
        total = total + avgbackflow
        counter = counter +1
        # avg for each 2 avg backflows
        if counter % interval == 0:
            avg = total / counter 
            #avgs.append(str(avg*100)+"%")
            avgs.append(avg)
            avg = 0
            counter = 0
            total =0
    if isAscending:
        return pd.DataFrame(quickSort(avgs, ascending=True))
    return pd.DataFrame(quickSort(avgs, ascending=False))

In [6]:
# plot data on scatter plot
def scatterPlotData(data, label, color):
    y = np.array(data)
    x = np.array(range(0, len(data)))
    plt.scatter(x, y, c=color)
    plt.xlabel("Avg at each interval")
    plt.ylabel(label)
    plt.show()

In [13]:
# get avg data for each 100 rows

avgtemps = getAvgData(df['Temp'], 99, isAscending=True)
avgpressure = getAvgData(df['Pressure'], 99, isAscending=True)
avgbackflows = getAvgData(df['Backflow'], 99, isAscending=False)

#scatterPlotData(avgpressure, "Avg Pressure", 'r')
#scatterPlotData(avgtemps, "Avg Temps", 'g')
#scatterPlotData(avgbackflows, "Avg Backflows", 'b')


In [14]:
dfconcat = pd.concat([avgtemps, avgpressure, avgbackflows],axis='columns', keys=['Avg Temps', 'Avg Pressure', 'Avg Backflow'])
df_data = dfconcat.swaplevel(axis='columns')
df_data

0                          
  Avg Temps Avg Pressure Avg Backflow
0 -0.409798    -5.148586    23.927475
1 -0.053333    -2.255152    23.438485
2 -0.016061    -1.402323     2.951515
3  0.040404    -0.792323     1.447172
4  0.061212    -0.136768     0.472222
5  0.077475     0.464242    -0.521616
6  0.078182     1.128283    -0.738485
7  0.100505     2.130707    -2.314646
8  0.110000     3.596768    -3.187374
9  0.200909     7.925253    -4.052828

In [9]:
# avg of avg backflow
dfconcat = pd.concat([getAvgAvgBackflows(avgbackflows.values, 2)],axis='columns', keys=['Avg Avg Backflow'])
df_avgAvgBF = dfconcat.swaplevel(axis='columns')
df_avgAvgBF
#Todo: Replicate data in Excel sheet

,0
,Avg Avg Backflow
0,-3.10855
1,-1.42090
2,-0.77100
3,2.21310
4,25.95455
